Packages

In [ ]:
import pennylane as qml
import numpy as np
from scipy.stats import norm

Parameters

In [ ]:
m = 5  # Number of qubits for discretization
n = 6  # Number of estimation qubits

# Discretization and scaling constants
M = 2 ** m                      # Total discretization points
N = 2 ** n                      # Applications of Q operator
x_max = 1.06                    # Maximum x value
x_min = 0.94                    # Minimum x value
c_s = np.power(3 * np.pi / N, 1 / 3)  # Scaling constant

# Rescale parameters
a = 2 * c_s / (M - 1)  # Scaling factor for f(i)
b = -c_s               # Offset for f(i)

# Generate x values and probabilities
x = np.linspace(-np.pi, np.pi, M)
p = np.array([norm(loc=0, scale=1).pdf(x_i) for x_i in x])  # Gaussian PDF
p /= np.sum(p)  # Normalize to form a probability distribution


def f(i):
    return a * i + b  # Linear transformation of index i

ancilla_qubit = m  # Ancilla qubit for rotations
a_qubits = range(m)  # Data qubits for discretization

def F():
    # Apply A - probability encoding unitary
    # Assume A(theta) is pre-trained and provided externally
    A(theta)

    # Apply controlled-Y rotations for R
    qml.RY(np.pi / 2, wires=ancilla_qubit)
    qml.RY(2 * b, wires=ancilla_qubit)
    for qubit in a_qubits:
        phi = 2 ** (m - qubit) * a
        qml.CRY(phi, wires=[qubit, ancilla_qubit])

QMC Circuit

In [ ]:
# Define qubit ranges
qubits = range(m)  # State preparation qubits
est_qubits = range(m + 1, n + m + 1)  # Estimation qubits

# Create a quantum device
n_qubits = m + n + 1  # Total qubits required
dev = qml.device("default.qubit", wires=n_qubits)

# Quantum Monte Carlo Circuit
@qml.qnode(dev)
def qmc():
    qml.quantum_monte_carlo(
        F,              # Oracle
        qubits,         # Probability qubits
        ancilla_qubit,  # Ancilla qubit
        est_qubits      # Estimation qubits
    )
    return qml.probs(est_qubits)  # Return probability estimates


# Execute the QMC circuit
p_est = qmc()

# Display results
print("Estimated Probabilities:", p_est)

# Analyze resource usage
expanded_tape = qmc.qtape.expand(depth=10)
specs = expanded_tape.specs

# Print circuit specs
print("Quantum Circuit Specifications:")
print(specs)
